<a href="https://colab.research.google.com/github/YairYarden/Deep-Learning/blob/sub-feature%2FQ3_VAE/Q3_VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Code is influenced by [This](TODO : Add reference) forum


# Imports

In [3]:
import numpy as np
from datetime import datetime 
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, SubsetRandomSampler

from torchvision import datasets, transforms

import matplotlib.pyplot as plt

from torch.utils.tensorboard import SummaryWriter

%load_ext tensorboard

# check if CUDA is available
is_gpu_available = torch.cuda.is_available()

if not is_gpu_available:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

DIR_PATH = '/content/drive/MyDrive/Deep_learning_05107255/ex3_316168061_313471526'


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
CUDA is not available.  Training on CPU ...


# Mount Google Drive

In [5]:
from google.colab import drive
drive.mount('/content/drive/')

drive_path = '/content/drive/MyDrive/Deep_learning_05107255/ex3_316168061_313471526'
print(drive_path)

Mounted at /content/drive/
/content/drive/MyDrive/Deep_learning_05107255/ex3_316168061_313471526


# Functions for data loading

In [6]:
def get_train_samplers(labels, n:int):
    """
    return 2 samplers of the train data, one for the labeled and one for the unlabeled data.

    Args:
        labels: array of all the labels in the dataset.
        n: number of labeled elements
    """
    n_classes = int(np.max(labels) + 1)
    n_per_class = n // n_classes


    labeled_indices = []
    unlabeled_indices = []

    for i in range(n_classes):
        curr_label_indices = np.where(labels==i)[0]
        labeled_indices.extend(curr_label_indices[:n_per_class].tolist())
        unlabeled_indices.extend(curr_label_indices[n_per_class:].tolist())

    return SubsetRandomSampler(labeled_indices), SubsetRandomSampler(unlabeled_indices)


def get_dataloaders(n, batch_size, num_workers=2):
    labeled_sampler, unlabeld_sampler = get_train_samplers(train_labels, n)
    train_labeled = DataLoader(train_dataset,
                               batch_size=batch_size,
                               sampler=labeled_sampler,
                               num_workers=num_workers,
                               pin_memory=True)
    

    train_unlabeled = DataLoader(train_dataset,
                                 batch_size=batch_size,
                                 sampler=unlabeld_sampler,
                                 num_workers=num_workers,
                                 pin_memory=True)
    
    test = DataLoader(test_dataset,
                        batch_size=batch_size,
                        num_workers=num_workers,
                        pin_memory=True)
    
    return train_labeled, train_unlabeled, test

# Load Data

In [7]:
# Define transform
transform = transforms.Compose([transforms.ToTensor()])

# Load Dataset
train_dataset = datasets.FashionMNIST(root=os.path.join(drive_path), 
                               train=True, 
                               transform=transform,
                               download=True,)

train_labels = np.array([label for _, label in train_dataset])

test_dataset = datasets.FashionMNIST(root=os.path.join(drive_path), 
                               train=False, 
                               transform=transform)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting /content/drive/MyDrive/Deep_learning_05107255/ex3_316168061_313471526/FashionMNIST/raw/train-images-idx3-ubyte.gz to /content/drive/MyDrive/Deep_learning_05107255/ex3_316168061_313471526/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting /content/drive/MyDrive/Deep_learning_05107255/ex3_316168061_313471526/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /content/drive/MyDrive/Deep_learning_05107255/ex3_316168061_313471526/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting /content/drive/MyDrive/Deep_learning_05107255/ex3_316168061_313471526/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /content/drive/MyDrive/Deep_learning_05107255/ex3_316168061_313471526/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting /content/drive/MyDrive/Deep_learning_05107255/ex3_316168061_313471526/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /content/drive/MyDrive/Deep_learning_05107255/ex3_316168061_313471526/FashionMNIST/raw

